<a href="https://colab.research.google.com/github/Rakib-hosen/Neural_network_from_scratch/blob/master/logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np
import tensorflow as tf
import zipfile
import os
import cv2
import random

In [7]:
!wget --no-check-certificate https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
                -O /tmp/horse-or-human.zip


# the Horse or Human validation dataset
!wget --no-check-certificate https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip \
                  -O /tmp/validation-horse-or-human.zip 


--2020-08-15 15:28:23--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.217.128, 172.217.193.128, 172.217.204.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.217.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘/tmp/horse-or-human.zip’

/tmp/horse-or-human 100%[===================>] 142.65M   213MB/s    in 0.7s    

2020-08-15 15:28:24 (213 MB/s) - ‘/tmp/horse-or-human.zip’ saved [149574867/149574867]

--2020-08-15 15:28:26--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.217.128, 172.217.204.128, 172.217.203.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.217.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Len

In [8]:
train_zip = '/tmp/horse-or-human.zip'
train_zip_ref  = zipfile.ZipFile(train_zip, 'r')
train_zip_ref.extractall('/tmp/train/')
train_zip_ref.close()
test_zip = '/tmp/validation-horse-or-human.zip'
test_zip_ref  = zipfile.ZipFile(test_zip, 'r')
test_zip_ref.extractall('/tmp/test/')
test_zip_ref.close()

In [9]:
train_dir = '/tmp/train/'
train_horses_dir = '/tmp/train/horses'
train_humans_dir = '/tmp/train/humans'
test_dir = '/tmp/train/'
test_horses_dir = '/tmp/test/horses'
test_humans_dir = '/tmp/test/horses'
img_size = 96
training_data = []
for i in os.listdir(train_horses_dir):
  im = cv2.imread(os.path.join(train_horses_dir,i),cv2.IMREAD_GRAYSCALE)
  img_array = cv2.resize(im, (img_size, img_size))
  training_data.append([img_array,0])
for i in os.listdir(train_humans_dir):
  im = cv2.imread(os.path.join(train_humans_dir,i),cv2.IMREAD_GRAYSCALE)
  img_array = cv2.resize(im, (img_size, img_size))
  training_data.append([img_array,1])
print(len(training_data))
testing_data = []
for i in os.listdir(test_horses_dir):
  im = cv2.imread(os.path.join(test_horses_dir,i),cv2.IMREAD_GRAYSCALE)
  img_array = cv2.resize(im, (img_size, img_size))
  testing_data.append([img_array,0])
for i in os.listdir(test_humans_dir):
  im = cv2.imread(os.path.join(test_humans_dir,i),cv2.IMREAD_GRAYSCALE)
  img_array = cv2.resize(im, (img_size, img_size))
  testing_data.append([img_array,1])

print(len(testing_data))

1027
256


In [10]:
training_data = random.sample(training_data, len(training_data))
testing_data = random.sample(testing_data, len(testing_data))
print(len(training_data))
X_train = []
Y_train = []
X_test = []
Y_test = []
for features,label in training_data:
  X_train.append(features)
  Y_train.append(label)
for features,label in testing_data:
  X_test.append(features)
  Y_test.append(label)

X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)


1027


In [11]:
X_train_flatten = X_train.reshape(X_train.shape[0],-1).T
X_test_flatten = X_test.reshape(X_test.shape[0],-1).T
Y_train = Y_train.reshape(Y_train.shape[0],-1).T
Y_test = Y_test.reshape(Y_test.shape[0],-1).T

print(X_train_flatten.shape)
print(Y_train.shape)
print(X_test_flatten.shape)
print(Y_test.shape)

(9216, 1027)
(1, 1027)
(9216, 256)
(1, 256)


In [12]:
X_test_flatten = X_test_flatten/255.0
X_train_flatten = X_train_flatten/255.0

In [13]:
def sigmoid(x):

  a = 1/(1+np.exp(-x))

  return a

In [17]:
print(sigmoid(2))
print(sigmoid(np.array([1,3,5,-1,-3,-5])))


0.8807970779778823
[0.73105858 0.95257413 0.99330715 0.26894142 0.04742587 0.00669285]


In [18]:
def initialization(dims):
  w = np.zeros((dims,1))
  b=0
 
  assert(w.shape == (dims, 1))
  assert(isinstance(b, float) or isinstance(b, int))
  return w,b

In [19]:
print(initialization(4))

(array([[0.],
       [0.],
       [0.],
       [0.]]), 0)


In [24]:
def  forward_and_backward_propagation(w, b, X, Y):

  m = X.shape[1]


  #forward propagation
  A = sigmoid(np.dot(w.T,X)+b)
  costs = -(np.sum(Y*np.log(A)+(1-Y)*np.log(1-A)))/m

  #backword propagation
  dw = np.dot(X,(A-Y).T)/m
  db = np.sum(A-Y)/m

  assert(dw.shape == w.shape)
  assert(db.dtype == float)
  costs = np.squeeze(costs)
  assert(costs.shape == ())

  gradients = {
      'dw' : dw,
      'db' : db
  }

  return gradients, costs

In [25]:
w, b, X, Y = np.array([[2.],[2.],[3]]), 2., np.array([[1.,2.,-1.],[3.,4.,-3.2],[4.,5.,1.]]), np.array([[1,0,1]])
print(X.shape)
grads, cost = forward_and_backward_propagation(w, b, X, Y)
print ("dw = " + str(grads["dw"]))
print ("db = " + str(grads["db"]))
print ("cost = " + str(cost))

(3, 3)
dw = [[0.98923485]
 [2.3655515 ]
 [1.34409849]]
db = 0.010765154806416377
cost = 10.810816320848764


In [26]:
def optimization(w, b, X, Y, num_iteration, learning_rate):

  costs = []

  for i in range(num_iteration):
    gradients, cost = forward_and_backward_propagation(w, b, X, Y)

    dw = gradients['dw']
    db = gradients['db']

    #update the parameters

    w = w - learning_rate * dw
    b = b - learning_rate * db

    if i%500 == 0:
      costs.append(cost)
      print(f'after {i} iterations cost is {cost}')

  parameters = {
      'w' : w,
      'b' : b
  }
  grads = {
      'dw' : dw,
      'db' : db
  }

  return parameters, grads, costs


In [27]:
optimization(w, b, X, Y,1000,0.01)

after 0 iterations cost is 10.810816320848764
after 500 iterations cost is 0.3935690666768905


({'b': 1.9294144470291523, 'w': array([[-0.12256011],
         [-1.14796625],
         [ 0.45515108]])},
 {'db': -0.03640968068210537, 'dw': array([[ 0.0806908 ],
         [ 0.01076744],
         [-0.02784872]])},
 [10.810816320848764, 0.3935690666768905])

In [28]:
def predict(w, b, X):

  m = X.shape[1]
  Y_prediction  = np.zeros((1, m))
  w = w.reshape(X.shape[0],1)

  A = sigmoid(np.dot(w.T,X)+b)

  for i in range(A.shape[1]):
    if A[0,i] >0.5:
      Y_prediction[0,i] = 1
    else:
      Y_prediction[0,i] = 0

  assert(Y_prediction.shape == (1, m))

  return Y_prediction

In [29]:
w = np.array([[0.1124579],[0.23106775]])
b = -0.3
X = np.array([[1.,-1.1,-3.2],[1.2,2.,0.1]])
print ("predictions = " + str(predict(w, b, X)))

predictions = [[1. 1. 0.]]


In [30]:
def model(X_train, Y_train, X_test, Y_test, num_of_iterations =1000, learning_rate=0.01):

  dims = X_train.shape[0]
  
  w, b = initialization(dims)
  parameters, grads, costs = optimization(w, b, X_train, Y_train, num_of_iterations, learning_rate)
  w = parameters['w']
  b = parameters['b']
  Y_train_prediction = predict(w, b, X_train)
  Y_test_prediction = predict(w, b, X_test)
  train_accuracy = 100 - np.mean(np.abs(Y_train_prediction - Y_train)) * 100
  test_accuracy = 100 - np.mean(np.abs(Y_test_prediction - Y_test)) * 100

  print(f'Train accuracy {train_accuracy}%')
  print(f'Test accuracy {test_accuracy}%')

  history = {
      'w' : w,
      'b' : b,
      'costs' : costs,
      'num_of_iterations' : num_of_iterations,
      'learning_rate' : learning_rate
  }


In [31]:
history  = model(X_train_flatten, Y_train, X_test_flatten, Y_test, num_of_iterations =1500,learning_rate=0.01)

after 0 iterations cost is 0.6931471805599453
after 500 iterations cost is 0.40759932064524074
after 1000 iterations cost is 0.20048407487956682
Train accuracy 95.71567672833496%
Test accuracy 50.0%
